# Product allocation

Formulation: https://github.com/opvious/examples/blob/main/sources/product-allocation.md

In [1]:
import io
import logging
import opvious
import pandas as pd

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s", datefmt="%H:%M:%S")

client = opvious.Client.from_environment()

In [2]:
demand_df = pd.read_csv(io.StringIO("""
location,tier,product,size,demand
Boston,T1,hoodie,M,50
Boston,T1,shirt,L,30
Boston,T2,shirt,L,25
Boston,T1,shirt,XL,20
Seattle,T1,hoodie,M,100
Seattle,T1,hoodie,L,75
Seattle,T2,hoodie,L,50
Seattle,T1,hoodie,XL,50
Seattle,T1,shirt,L,10
""")).set_index(["location", "product", "size", "tier"])
demand_df

demand
location product size tier        
Boston   hoodie  M    T1        50
         shirt   L    T1        30
                      T2        25
                 XL   T1        20
Seattle  hoodie  M    T1       100
                 L    T1        75
                      T2        50
                 XL   T1        50
         shirt   L    T1        10

In [3]:
supply_df = pd.read_csv(io.StringIO("""
product,size,supply
hoodie,M,100
hoodie,L,50
shirt,L,50
shirt,XL,10
""")).set_index(["product", "size"])
supply_df

supply
product size        
hoodie  M        100
        L         50
shirt   L         50
        XL        10

In [4]:
product_df = pd.read_csv(io.StringIO("""
product,min_allocation,diversity
hoodie,100,2
shirt,10,2
""")).set_index(["product"])
product_df

,min_allocation,diversity
product,,
hoodie,100,2
shirt,10,2


In [5]:
response = await client.run_solve(
    specification=opvious.RemoteSpecification.example("product-allocation"),
    parameters={
        "demand": demand_df["demand"],
        "demandValue": {"T1": 1, "T2": 0.8},
        "minAllocation": product_df["min_allocation"],
        "productDiversity": product_df["diversity"],
        "supply": supply_df["supply"],
        "maxTotalAllocation": 500
    },
    assert_feasible=True
)

22:35:53 INFO Validated inputs. [parameters=20]
22:35:53 INFO Solving problem... [columns=40, rows=75]
22:35:53 INFO Solve completed with status OPTIMAL. [objective=206]


In [6]:
response.outputs.variable("allocation")

value
location product size demandTier       
Seattle  hoodie  M    T1            100
                 L    T1             50
Boston   shirt   L    T1             30
                 XL   T1             10
                 L    T2             20

In [7]:
response.summary.constraints

,row_count,row_sprs,column_count,column_sprs,weight_count,weight_min,weight_max,weight_mean,weight_stddev,weight_sprs,reify_ms
label,,,,,,,,,,,
atLeastMinAllocation,4,0.0,28,0.000000,28,-100,1,-7.000000,26.423895,1.386294,6
atLeastMinDiversity,4,0.0,16,0.000000,16,-2,1,0.250000,1.341641,1.386294,3
atMostDemand,24,0.0,24,0.000000,24,1,1,1.000000,0.000000,3.178054,19
atMostMaxTotalAllocation,1,0.0,24,0.000000,24,1,1,1.000000,0.000000,0.000000,1
atMostSupply,6,0.0,24,0.000000,24,1,1,1.000000,0.000000,1.791759,13
productActivation,12,0.0,16,0.000000,24,-1,1,0.000000,1.021508,2.079442,1
sizeActivation,12,0.0,32,0.117783,32,-1,100,12.375000,28.581237,2.602690,13
sizeDeactivation,12,0.0,36,0.000000,36,-1,1,-0.333333,0.956183,2.484907,2
